# IoT - Human Detection Notebook 📹

Giovanni Baccichet (10851745) - Politecnico di Milano

## Libraries 📔

### Libraries: Importing and Visualizing the Dataset

In [1]:
import pandas as pd 
import glob
import matplotlib.pyplot as plt
import numpy as np
from numpy import array
import seaborn as sns
import time

ModuleNotFoundError: No module named 'matplotlib'

### Libraries: Random Forest Training and Optimization

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import completeness_score, homogeneity_score, v_measure_score

import pickle

## Pre-Processing 🧼

In [ ]:
path = '' # Dataset in the notebook's same folder
all_files = glob.glob(path + "dataset-1-rtsp-src=camera.csv")

# Merge all 
df = pd.concat((pd.read_csv(f, encoding='cp1252', header=0) for f in all_files))
df.columns = df.columns.str.replace(r' ','_', regex=True)

In [ ]:
print(df.shape)

In [ ]:
df.head()